In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from collections import OrderedDict

<h2>Function 1: Get a Show's Info</h2>

In [103]:
def get_all_show_info(url):
    
    #URL request/response cycle  
    response = requests.get(url)
    if response.status_code == 200:
        print("Show found! ", url)
    else:
        print("Failure")
    results_page = BeautifulSoup(response.content, 'lxml')
    
    all_show_info = list()
    
    #Part 1 -- Get the date 
    date_block = results_page.find_all('div', class_="dateBlock")
    date_block_as_list = ''.join(map(str, date_block)).split(">")

    #Get the month
    try:
        month = date_block_as_list[2].rstrip("</span")
    except:
        pass

    #Get the day
    try:
        day = date_block_as_list[4].rstrip("</span")
    except:
        pass

    #Get the year
    try:
        year = date_block_as_list[6].rstrip("</span")
    except:
        pass
    #Create a tuple of the date info and add it to the all_show_info list
    date_tuple = (month, day, year)
    all_show_info.append(date_tuple)
    
    
    #Part 2 -- Get the location info
    try:
        header_info = results_page.find('h1').find_all('span')
    except:
        pass
    
    try:
        location_info = header_info[3].find('span').get_text()
    except:
        pass
    
    try:
        venue, city, state, country = location_info.split(",")
    except:
        pass
    
    location_tuple = (venue, city, state, country) 
    all_show_info.append(location_tuple)
        
    #Part 3 -- Get the songs
    setlist = list()
    try:
        messy_songs = results_page.find_all('div', class_="songPart")
    except:
        pass
    
    try:
        for song in messy_songs:
            song_title = song.find('a').get_text()
            setlist.append(song_title)

        setlist_tuple = tuple(setlist)
        all_show_info.append(setlist_tuple)
    except:
        pass
    
    return all_show_info

In [105]:
def get_all_show_info_improved(url):
    
    #URL request/response cycle  
    response = requests.get(url)
    if response.status_code == 200:
        print("Show found!")
    else:
        print("Failure")
    results_page = BeautifulSoup(response.content, 'lxml')
    
    all_show_info = list()
    
    #Part 1 -- Get the date 
    try:
        full_date = results_page.find_all('div', class_="breadCrumbBar")[0].find_all('span')[-1].get_text().rstrip('Setlist')
        month, day, year = full_date.strip().replace(",","").split(" ")
    except:
        pass
    
    #Create a tuple of the date info and add it to the all_show_info list
    try:
        date_tuple = (month, day, year)
        all_show_info.append(date_tuple)
    except:
        pass
    
    #Part 2 -- Get the location info
    try:
        header_info = results_page.find('h1').find_all('span')
    except:
        pass
    
    try:
        location_info = header_info[3].find('span').get_text()
    except:
        pass
    
    try:
        venue, city, state, country = location_info.split(",")
    except:
        pass
    
    try:
        location_tuple = (venue, city, state, country) 
        all_show_info.append(location_tuple)
    except:
        pass
        
    #Part 3 -- Get the songs
    setlist = list()
    try:
        set_list_list = results_page.find_all('a', 'songLabel')
        for song in set_list_list:
            setlist.append(song.get_text())
   
        setlist_tuple = tuple(setlist)
        all_show_info.append(setlist_tuple)
    
    except:
        pass
    
    return all_show_info

<h4> Test out the function -- get_all_show_info</h4>

In [106]:
show_data = get_all_show_info_improved('https://www.setlist.fm/setlist/grateful-dead/1984/san-francisco-civic-auditorium-san-francisco-ca-53d607d9.html')
print(show_data)


Show found!
[('December', '31', '1984'), ('San Francisco  Civic Auditorium', ' San Francisco', ' CA', ' USA'), ('Shakedown Street', 'Minglewood Blues', 'Peggy-O', 'Jack Straw', 'Bird Song', 'Hell in a Bucket', "Don't Ease Me In", 'Sugar Magnolia', 'Scarlet Begonias', 'Fire on the Mountain', 'Man Smart, Woman Smarter', 'Drums', 'Space', 'Spanish Jam', 'The Wheel', 'Throwing Stones', 'Turn On Your Love Light', "Gimme Some Lovin'", "Uncle John's Band", 'Around and Around', "It's All Over Now, Baby Blue")]


In [107]:
show_data_output = get_all_show_info('https://www.setlist.fm/setlist/grateful-dead/1984/san-francisco-civic-auditorium-san-francisco-ca-53d607d9.html')


Show found!  https://www.setlist.fm/setlist/grateful-dead/1984/san-francisco-civic-auditorium-san-francisco-ca-53d607d9.html


<h2>Function 2: Add the first show's info AND songs to a new dataframe</h2>

In [108]:
def add_all_show_info_to_df(show_data_output):
    from collections import OrderedDict
    
    try:
        show_month = show_data_output[0][0]
    except:
        pass
    
    try:
        show_day = show_data_output[0][1]
    except:
        pass
    
    try:
        show_year = show_data_output[0][2]
    except:
        pass
    
    try:
        show_venue = show_data_output[1][0] 
    except:
        pass
    
    try:
        show_city = show_data_output[1][1]
    except:
        pass
    
    try:
        show_state = show_data_output[1][2]
    except:
        pass
    
    try:
        show_country = show_data_output[1][3]
    except:
        pass
    
    show_as_index = "%s-%s-%s" % (show_year, show_month, show_day)
    
    try:
        show_dict = {
    'Index': [show_as_index], 'Year': [show_year], 'Month': [show_month], 'Day': [show_day],
    'Venue': [show_venue], 'City': [show_city], "State": [show_state], "Country": [show_country]}
    except:
        pass
    
    try:
        set_list = show_data_output[2]
        song_dict = OrderedDict() #Create an ordered dict to hold the songs/setlist

        for number, song_title in enumerate(set_list):
            key = number+1 #This ensures the track listing starts at 1 instead of 0
            pretty_key = "Song" + str(key) 
            value = song_title
            song_dict[pretty_key] = value #Add each key, value to the ordered dict where key=Song# and value = track name

        show_dict.update(song_dict) #Append/add the songs to the show dict
    
    except:
        pass
    
    try:
        show_dict
    except:
        column_list = ['Index','Year','Month','Day','Venue','City','State','Country' ]
        df = pd.DataFrame(columns = column_list)
    else: 
        df = pd.DataFrame(show_dict)
    
    return df

In [109]:
first_show_df = add_all_show_info_to_df(show_data_output)
first_show_df

,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song12,Song13,Song14,Song15,Song16,Song17,Song18,Song19,Song20,Song21
0,1984-Dec-31,1984,Dec,31,San Francisco Civic Auditorium,San Francisco,CA,USA,Shakedown Street,Minglewood Blues,...,Drums,Space,Spanish Jam,The Wheel,Throwing Stones,Turn On Your Love Light,Gimme Some Lovin',Uncle John's Band,Around and Around,"It's All Over Now, Baby Blue"


In [110]:
second_show = get_all_show_info_improved('https://www.setlist.fm/setlist/grateful-dead/1995/deer-creek-music-center-noblesville-in-bd6bdea.html')
second_show_df = add_all_show_info_to_df(second_show)
second_show_df

Show found!


,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song9,Song10,Song11,Song12,Song13,Song14,Song15,Song16,Song17,Song18
0,1995-July-2,1995,July,2,Deer Creek Music Center,Noblesville,IN,USA,Here Comes Sunshine,Walkin' Blues,...,Scarlet Begonias,Fire on the Mountain,Victim or the Crime,It's All Too Much,New Speedway Boogie,Drums,Space,Attics of My Life,Sugar Magnolia,Quinn the Eskimo (The Mighty Quinn)


In [111]:
combo_df = first_show_df.append(second_show_df, ignore_index=True, sort=False)
combo_df

,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song12,Song13,Song14,Song15,Song16,Song17,Song18,Song19,Song20,Song21
0,1984-Dec-31,1984,Dec,31,San Francisco Civic Auditorium,San Francisco,CA,USA,Shakedown Street,Minglewood Blues,...,Drums,Space,Spanish Jam,The Wheel,Throwing Stones,Turn On Your Love Light,Gimme Some Lovin',Uncle John's Band,Around and Around,"It's All Over Now, Baby Blue"
1,1995-July-2,1995,July,2,Deer Creek Music Center,Noblesville,IN,USA,Here Comes Sunshine,Walkin' Blues,...,It's All Too Much,New Speedway Boogie,Drums,Space,Attics of My Life,Sugar Magnolia,Quinn the Eskimo (The Mighty Quinn),NaN,NaN,NaN


<h2>Function 3: Combo of Functions 1 and 2, Get Show's Info and Create a DF of it</h2>

In [112]:
def get_show_info_and_make_df(show_url):
    show_info_as_list = get_all_show_info_improved(str(show_url))
    show_as_df = add_all_show_info_to_df(show_info_as_list)
    return show_as_df
    


In [113]:
get_show_info_and_make_df("https://www.setlist.fm/setlist/billy-joel/1999/raleigh-entertainment-and-sports-arena-raleigh-nc-33dd2055.html")

Show found!


,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song18,Song19,Song20,Song21,Song22,Song23,Song24,Song25,Song26,Song27
0,1999-December-21,1999,December,21,Raleigh Entertainment and Sports Arena,Raleigh,NC,USA,Only the Good Die Young,Movin' Out (Anthony's Song),...,Uptown Girl,Suspicious Minds,Sunshine of Your Love,Good Times Bad Times,We Didn't Start the Fire,It's Still Rock and Roll to Me,You May Be Right,Scenes From an Italian Restaurant,Silent Night,Piano Man


<h2>Function 4: Combine the first and subsequent show DFs</h2>

In [114]:
def add_more_shows(first_show_df, second_show_df):
    combo_df = first_show_df.append(second_show_df, ignore_index=True, sort=False)
    return combo_df

In [115]:
combo_df = add_more_shows(first_show_df, second_show_df)
combo_df

,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song12,Song13,Song14,Song15,Song16,Song17,Song18,Song19,Song20,Song21
0,1984-Dec-31,1984,Dec,31,San Francisco Civic Auditorium,San Francisco,CA,USA,Shakedown Street,Minglewood Blues,...,Drums,Space,Spanish Jam,The Wheel,Throwing Stones,Turn On Your Love Light,Gimme Some Lovin',Uncle John's Band,Around and Around,"It's All Over Now, Baby Blue"
1,1995-July-2,1995,July,2,Deer Creek Music Center,Noblesville,IN,USA,Here Comes Sunshine,Walkin' Blues,...,It's All Too Much,New Speedway Boogie,Drums,Space,Attics of My Life,Sugar Magnolia,Quinn the Eskimo (The Mighty Quinn),NaN,NaN,NaN


<h3>##################################################################</h3>

<h2>Function 5: Access any artists landing page and get the link to their setlists</h2>

In [116]:
def find_an_artist():    
    artist_name = input(" Please enter the name of an artist")
    if ' ' in artist_name:
        artist_name = artist_name.replace(" ", "%20")    
    url = "https://www.setlist.fm/search?query=" + artist_name
    print(url)
    print('')
    response = requests.get(url)
    if response.status_code == 200:
        print("Artist Found!")
        print('')
    else:
        print("Failed to connect with website")
    results_page = BeautifulSoup(response.content, 'lxml')

    left_rail = results_page.find('div', class_="col-xs-12 col-md-3")
    more_from_artist = left_rail.find('div', class_="col-xs-12")
    setlist_page_link = 'https://www.setlist.fm/'+more_from_artist.find('a').get('href')
    return setlist_page_link


<h4> Test out the function</h4>

In [117]:
find_an_artist()

 Please enter the name of an artistBilly Joel
https://www.setlist.fm/search?query=Billy%20Joel

Artist Found!



'https://www.setlist.fm/setlists/billy-joel-7bd6be40.html'

<h2>Function 6: Grab all show URLs and add to a list</h2>

In [118]:
def create_list_of_artist_sets(setlist_link):
    response = requests.get(setlist_link)
    if response.status_code == 200:
        print("Setlists found!")
    else:
        print("Failure. BOOOOOOOO")
    results_page = BeautifulSoup(response.content, 'lxml')

    all_shows_urls = list()
    show_blocks = results_page.find_all('div', class_="col-xs-12 setlistPreview vevent")
    number_of_shows = len(show_blocks)

    counter = 0
    for show in show_blocks:
        show_url = 'https://www.setlist.fm/' + show_blocks[counter].find('h2').find('a').get('href').strip("../")
        all_shows_urls .append(show_url)
        counter +=1 
    
    return all_shows_urls

In [119]:
create_list_of_artist_sets('https://www.setlist.fm/setlists/billy-joel-7bd6be40.html')

Setlists found!


['https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-6b97be66.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-53963335.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/bbandt-field-winston-salem-nc-4396ab87.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-23e91c6f.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/kauffman-stadium-kansas-city-mo-13e9dd61.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/wrigley-field-chicago-il-53e8535d.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-73e8d2e9.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/fenway-park-boston-ma-23e8a49f.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/fenway-center-boston-ma-7be8d620.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/citizens-bank-park-philadelphia-pa-beb2536.html']

<h2>Function 7: Combine the previous two functions. Enter an artist, find their setlist page and add their first page of show URLs to a list</h2>

In [120]:
def find_artist_and_setlists():
    artist_setlist_page = find_an_artist()
    list_of_setlists = create_list_of_artist_sets(artist_setlist_page)
    return list_of_setlists

In [121]:
find_artist_and_setlists()

 Please enter the name of an artistBilly Joel
https://www.setlist.fm/search?query=Billy%20Joel

Artist Found!

Setlists found!


['https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-6b97be66.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-53963335.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/bbandt-field-winston-salem-nc-4396ab87.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-23e91c6f.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/kauffman-stadium-kansas-city-mo-13e9dd61.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/wrigley-field-chicago-il-53e8535d.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-73e8d2e9.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/fenway-park-boston-ma-23e8a49f.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/fenway-center-boston-ma-7be8d620.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/citizens-bank-park-philadelphia-pa-beb2536.html']

In [122]:
bj_first_ten_shows = find_artist_and_setlists()

 Please enter the name of an artistBilly Joel
https://www.setlist.fm/search?query=Billy%20Joel

Artist Found!

Setlists found!


In [123]:
bj_first_ten_shows

['https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-6b97be66.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-53963335.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/bbandt-field-winston-salem-nc-4396ab87.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-23e91c6f.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/kauffman-stadium-kansas-city-mo-13e9dd61.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/wrigley-field-chicago-il-53e8535d.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/madison-square-garden-new-york-ny-73e8d2e9.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/fenway-park-boston-ma-23e8a49f.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/fenway-center-boston-ma-7be8d620.html',
 'https://www.setlist.fm/setlist/billy-joel/2018/citizens-bank-park-philadelphia-pa-beb2536.html']

In [124]:
first_bj_show_url = bj_first_ten_shows[0] 
show_data_df_output = get_show_info_and_make_df(first_bj_show_url)
show_data_df_output
# for showlink in bj_first_ten_shows:
#     show_as_df = get_all_show_info(first_bj_show)
#     add_more_shows(first_bj_show_df, show_as_df)

Show found!


,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song15,Song16,Song17,Song18,Song19,Song20,Song21,Song22,Song23,Song24
0,2018-November-10,2018,November,10,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,Walk This Way,Bohemian Rhapsody,Only the Good Die Young,Nessun dorma,Scenes From an Italian Restaurant,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right


In [125]:
for show_url in bj_first_ten_shows:
    show_df = get_show_info_and_make_df(show_url)
    combo_frame = add_more_shows(show_data_df_output, show_df)
    show_data_df_output = combo_frame 

#add_more_shows(show_data_df_output,bj_first_ten_shows[1])
show_data_df_output

Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!


,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song21,Song22,Song23,Song24,Song25,Song26,Song27,Song28,Song29,Song30
0,2018-November-10,2018,November,10,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-November-10,2018,November,10,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-October-27,2018,October,27,Madison Square Garden,New York,NY,USA,Miami 2017 (Seen the Lights Go Out on Broadway),Big Shot,...,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN
3,2018-October-13,2018,October,13,BB&T Field,Winston Salem,NC,USA,Big Shot,My Life,...,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-September-30,2018,September,30,Madison Square Garden,New York,NY,USA,Big Shot,Pressure,...,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,Rock and Roll,NaN,NaN,NaN,NaN
5,2018-September-21,2018,September,21,Kauffman Stadium,Kansas City,MO,USA,Big Shot,My Life,...,Scenes From an Italian Restaurant,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN
6,2018-September-7,2018,September,7,Wrigley Field,Chicago,IL,USA,Big Shot,My Life,...,Scenes From an Italian Restaurant,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN
7,2018-August-23,2018,August,23,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,Uptown Girl,It's Still Rock and Roll to Me,Summer of '69,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
8,2018-August-10,2018,August,10,Fenway Park,Boston,MA,USA,Big Shot,My Life,...,Scenes From an Italian Restaurant,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN
9,2018-August-9,2018,August,9,Fenway Center,Boston,MA,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [476]:
show_data_df_output

,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song21,Song22,Song23,Song24,Song25,Song26,Song27,Song28,Song29,Song30
0,2018-Nov-10,2018,Nov,10,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-Nov-10,2018,Nov,10,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-Oct-27,2018,Oct,27,Madison Square Garden,New York,NY,USA,Miami 2017 (Seen the Lights Go Out on Broadway),Big Shot,...,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN
3,2018-Oct-13,2018,Oct,13,BB&T Field,Winston Salem,NC,USA,Big Shot,My Life,...,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-Se-30,2018,Se,30,Madison Square Garden,New York,NY,USA,Big Shot,Pressure,...,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,Rock and Roll,NaN,NaN,NaN,NaN
5,2018-Se-21,2018,Se,21,Kauffman Stadium,Kansas City,MO,USA,Big Shot,My Life,...,Scenes From an Italian Restaurant,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN
6,2018-Se-7,2018,Se,7,Wrigley Field,Chicago,IL,USA,Big Shot,My Life,...,Scenes From an Italian Restaurant,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN
7,2018-Aug-23,2018,Aug,23,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,Uptown Girl,It's Still Rock and Roll to Me,Summer of '69,You May Be Right,NaN,NaN,NaN,NaN,NaN,NaN
8,2018-Aug-10,2018,Aug,10,Fenway Park,Boston,MA,USA,Big Shot,My Life,...,Scenes From an Italian Restaurant,Piano Man,We Didn't Start the Fire,Uptown Girl,It's Still Rock and Roll to Me,You May Be Right,NaN,NaN,NaN,NaN
9,2018-Aug-9,2018,Aug,9,Fenway Center,Boston,MA,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [487]:
set_list_page_links = list()
pagenumber = 2
for num in range(143):
    show_pages = 'https://www.setlist.fm/setlists/billy-joel-7bd6be40.html?page=' + str(pagenumber)
    set_list_page_links.append(show_pages)
    pagenumber += 1

In [491]:
set_list_page_links[0:5]

['https://www.setlist.fm/setlists/billy-joel-7bd6be40.html?page=2',
 'https://www.setlist.fm/setlists/billy-joel-7bd6be40.html?page=3',
 'https://www.setlist.fm/setlists/billy-joel-7bd6be40.html?page=4',
 'https://www.setlist.fm/setlists/billy-joel-7bd6be40.html?page=5',
 'https://www.setlist.fm/setlists/billy-joel-7bd6be40.html?page=6']

In [492]:
full_show_list = list()
for page_link in set_list_page_links:
   full_show_list.extend(create_list_of_artist_sets(page_link))


Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists found!
Setlists

In [512]:
zero_to_100 = full_show_list[:100]


1423

In [514]:
for show_url in full_show_list:
    show_df = get_show_info_and_make_df(show_url)
    combo_frame = add_more_shows(show_data_df_output, show_df)
    show_data_df_output = combo_frame 

Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show

Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show

Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!
Show found!


In [519]:
show_data_df_output.head(20)
df = 

,Index,Year,Month,Day,Venue,City,State,Country,Song1,Song2,...,Song28,Song29,Song30,Song31,Song32,Song33,Song34,Song35,Song36,Song37
0,2018-Nov-10,2018,Nov,10,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-Nov-10,2018,Nov,10,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-Oct-27,2018,Oct,27,Madison Square Garden,New York,NY,USA,Miami 2017 (Seen the Lights Go Out on Broadway),Big Shot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-Oct-13,2018,Oct,13,BB&T Field,Winston Salem,NC,USA,Big Shot,My Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-Se-30,2018,Se,30,Madison Square Garden,New York,NY,USA,Big Shot,Pressure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018-Se-21,2018,Se,21,Kauffman Stadium,Kansas City,MO,USA,Big Shot,My Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018-Se-7,2018,Se,7,Wrigley Field,Chicago,IL,USA,Big Shot,My Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2018-Aug-23,2018,Aug,23,Madison Square Garden,New York,NY,USA,Big Shot,My Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018-Aug-10,2018,Aug,10,Fenway Park,Boston,MA,USA,Big Shot,My Life,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2018-Aug-9,2018,Aug,9,Fenway Center,Boston,MA,USA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
